In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
#For google colab we mount google drive 
from google.colab import drive 
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
URL = "https://recruitment.aimtechnologies.co/ai-tasks"
tweets_ids_csv = "/content/drive/MyDrive/AIM Technologies Task/dialect_dataset.csv"

In [ ]:
# Read the given dataset which contain the Ids and the labels
tweets_ids = pd.read_csv(tweets_ids_csv)
tweets_ids.head()

,id,dialect
0,1175358310087892992,IQ
1,1175416117793349632,IQ
2,1175450108898565888,IQ
3,1175471073770573824,IQ
4,1175496913145217024,IQ


In [ ]:
tweets_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  int64 
 1   dialect  458197 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [ ]:
def data_fetch(df,col, chunck_size):
    '''
    INPUT: dataframe, column_of_ids, Chunk_size(Not exceed 1000)
    OUTPUT: Tweets
    Description: Making data fetch function to call the API with the specified Ids
    '''
    df = df.astype(str)
    tweets = np.array([])
    lst = df[col].tolist()
    chunks = [lst[i:i + 1000] for i in range(0, len(lst), chunck_size)]
    for chunck in tqdm(chunks):
        json_chunck = json.dumps(chunck)
        r = requests.post(URL, data = json_chunck)
        tweets_chunck = r.json()
        tweets_chunck_list = list(tweets_chunck.values())
        tweets = np.append(tweets, tweets_chunck_list)
    tweets = tweets.flatten()
    return tweets

In [ ]:
returned_tweets = data_fetch(tweets_ids,'id', 1000)

100%|██████████| 459/459 [10:42<00:00,  1.40s/it]


In [ ]:
# make a copy of the original dataframe to append
fetched_tweets=tweets_ids.copy()
fetched_tweets.head()

,id,dialect
0,1175358310087892992,IQ
1,1175416117793349632,IQ
2,1175450108898565888,IQ
3,1175471073770573824,IQ
4,1175496913145217024,IQ


In [ ]:
# append the tweets column in final_df dataframe
fetched_tweets['tweets'] = returned_tweets

In [ ]:
fetched_tweets.head()

,id,dialect,tweets
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [ ]:
# save the final_df to csv file
fetched_tweets.to_csv("tweets_dataset.csv")

In [ ]:
df=pd.read_csv("tweets_dataset.csv", lineterminator='\n')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  458197 non-null  int64 
 1   id          458197 non-null  int64 
 2   dialect     458197 non-null  object
 3   tweets      458197 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.0+ MB


In [ ]:
#copy the final tweets dataframe to drive to load for the rest of the task
!cp tweets_dataset.csv /content/drive/MyDrive/